# Coletando todos os dados disponíveis sobre cada avaliação individual

## Importando as bibliotecas

In [0]:
import re
import os
import csv
import itertools

from pprint import pprint

## Importando os arquivos

In [0]:
!unzip -q "/content/drive/My Drive/PBICT-UEA-2019 2020/arquivos_de_mineracao/codebench.zip"

## Definindo o ponto raíz do diretório que será usado

In [0]:
# root = "../codebench/{semester}/{class}/assessments/{assessment}.data"
# root = "../codebench/"
root = "codebench/"

## Lendo o arquivo e extraindo os dados

In [0]:
def read_file(semester, class_id, assessment, path):
    title, start, end, weight, total_exercises, type_assessment, exercises_number = None, None, None, None, None, None, None
    exercises_numbers = []
    with open(path, "r") as fl:
        for line in fl.readlines():
            # print(line)

            # ---- assessment title: 
            # ---- start: 2016-08-11 10:57
            # ---- end: 2016-08-11 11:55
            # ---- weight: 1
            # ---- total_exercises: 2
            # ---- type: homework

            
            if title is None: title = re.search(r"assessment title: (.+)", line)
            if start is None: start = re.search(r"start: (.+)", line)
            if end is None: end = re.search(r"end: (.+)", line)
            if weight is None: weight = re.search(r"weight: (.+)", line)
            if total_exercises is None: total_exercises = re.search(r"total_exercises: (.+)", line)
            if type_assessment is None: type_assessment = re.search(r"type: (.+)", line)
            exercises_number = exercises_number = re.search("exercise\s\d+\:\s(.+)", line)
            
            if title is not None and not isinstance(title, str):
                title = title.group(1)
            elif start is not None and not isinstance(start, str):
                start = start.group(1)
            elif end is not None  and not isinstance(end, str):
                end = end.group(1)
            elif weight is not None  and not isinstance(weight, str):
                weight = weight.group(1)
            elif total_exercises is not None  and not isinstance(total_exercises, str):
                total_exercises = total_exercises.group(1)
            elif type_assessment is not None  and not isinstance(type_assessment, str):
                type_assessment = type_assessment.group(1)
            elif exercises_number is not None:
                exercises_number = exercises_number.group(1).split(' or ')
                exercises_numbers.append(exercises_number)


        merged = list(itertools.chain.from_iterable(exercises_numbers))

        merged = ', '.join(merged)
        registry = [semester, class_id, assessment.replace('.data', ''), start, end, weight, total_exercises, type_assessment, title, merged]
        # print(registry)
    return registry


## Chamando a função extratora para cada avaliação existente

In [0]:
dataset = [['semester', 'class', 'assessment',  'start_date', 'end_date', 'assessment_weight', 'total_exercises', 'assessment_type', 'title', 'exercises_number']]
for semester in os.listdir(root):
    build_path = os.path.join(root, semester)
    for class_id in os.listdir(build_path):
        new_path = os.path.join(build_path, class_id, "assessments/")
        for assessment in os.listdir(new_path):
            full_path = os.path.join(new_path, assessment)
            # print(full_path)
            registry = read_file(semester, class_id, assessment, full_path)
            dataset.append(registry)
#pprint(dataset)

## Gerando um CSV com os dados extraídos

In [0]:

with open('/content/drive/My Drive/PBICT-UEA-2019 2020/arquivos_de_mineracao/datasets_csv/informacoes_sobre_os_assessments_completo.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',')
    for registry in dataset:
        print(registry)
        filewriter.writerow(registry)